In [1]:
import pandas as pd
import datetime
import urllib
import numpy as np
from io import BytesIO
from io import StringIO

In [2]:
def get_data(symbol,start,end):
    base_url = 'https://query1.finance.yahoo.com/v7/finance/download/'
    #EJ Symbol: ^GSPC
    base_url = base_url+symbol+'?%s'
    payload = urllib.parse.urlencode({'period1': start, 'period2': end, 'interval': '1d', 'events':'history'})

    req = urllib.request.urlopen(base_url % payload)
    req_read = req.read()
    file_download = BytesIO(req_read)

    df = pd.read_csv(file_download)

    df['Close'].replace('', np.nan, inplace=True)
    df.dropna(subset=['Close'], inplace=True)
    
    df['Symbol'] = symbol
    df['actualizacion'] = datetime.datetime.now()

    return df

In [16]:
def upload_azure():
    #Verificar si la ruta existe.
    #symbols = ['^GSPC','GC=F','BG','SI=F','^DJI','GLEN.L','ADM']

    data = pd.DataFrame()


    symbol = 'CNYUSD=X'
    max_date = datetime.datetime(2010, 1, 4, 0, 0) #'2010-01-04'

    start_date = datetime.datetime.combine(max_date, datetime.datetime.min.time()) + datetime.timedelta(days=1)
    end_date = datetime.datetime.now() - datetime.timedelta(days=1)

    date_now_timestamp = str(datetime.datetime.timestamp(start_date)).split('.')
    start = date_now_timestamp[0]

    date_now_timestamp = str(datetime.datetime.timestamp(end_date)).split('.')
    end = date_now_timestamp[0]

    symbol_data = get_data(symbol,start,end)

    if(symbol_data.empty):
        return '{"Result":"False"}'

    file_download = symbol_data.to_csv(header=True,index=False,encoding='utf-8-sig')        
    #name = 'YAHOO_FINANCE.csv'
    
    return file_download

In [17]:
yah = upload_azure()

In [18]:
yah_csv = StringIO(yah)
yah_pd = pd.read_csv(yah_csv, sep=',')

In [19]:
yah_pd.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,actualizacion
0,2010-01-05,0.146692,0.146692,0.146692,0.146692,0.146692,0.0,CNYUSD=X,2020-11-19 09:08:23.908652
1,2010-01-06,0.146692,0.146692,0.146664,0.146664,0.146664,0.0,CNYUSD=X,2020-11-19 09:08:23.908652
2,2010-01-07,0.146664,0.146675,0.146664,0.146675,0.146675,0.0,CNYUSD=X,2020-11-19 09:08:23.908652
3,2010-01-08,0.146671,0.146681,0.146671,0.146681,0.146681,0.0,CNYUSD=X,2020-11-19 09:08:23.908652
4,2010-01-11,0.146703,0.146703,0.146703,0.146703,0.146703,0.0,CNYUSD=X,2020-11-19 09:08:23.908652


In [20]:
yah_pd.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,actualizacion
2826,2020-11-12,0.150811,0.151371,0.150775,0.150811,0.150811,0.0,CNYUSD=X,2020-11-19 09:08:23.908652
2827,2020-11-13,0.151272,0.151582,0.150934,0.151272,0.151272,0.0,CNYUSD=X,2020-11-19 09:08:23.908652
2828,2020-11-16,0.151391,0.152427,0.151391,0.151391,0.151391,0.0,CNYUSD=X,2020-11-19 09:08:23.908652
2829,2020-11-17,0.151888,0.152742,0.151879,0.151888,0.151888,0.0,CNYUSD=X,2020-11-19 09:08:23.908652
2830,2020-11-18,0.152553,0.153114,0.152351,0.152553,0.152553,0.0,CNYUSD=X,2020-11-19 09:08:23.908652


In [21]:
yah_pd.to_csv('YAHOO_FINANCE.csv',index=False)

In [9]:
#Upload to Azure Blob Storage and Execute Manual Trigger.
import pyodbc
pyodbc.drivers()

['ODBC Driver 17 for SQL Server']

In [10]:
server = 'grainpredictive.database.windows.net'
database = 'gpi'
username = 'gpi'
password = 'Cmi@2019$A'   
driver= '{ODBC Driver 17 for SQL Server}'

In [11]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

In [12]:
query="""SELECT TOP (1000) [Symbol]
      ,MAX([Date])
      ,MAX([actualizacion])
  FROM [ST_YAHOO].[HISTORICAL_MARKET]
  GROUP BY [Symbol]"""

In [23]:
data = pd.read_sql(query, cnxn)
data

,Symbol,,
0,^DJI,2020-11-17,2020-11-19 08:01:52.987
1,^GSPC,2020-11-17,2020-11-19 08:01:53.080
2,000001.SS,2020-11-18,2020-11-19 08:52:04.010
3,ADM,2020-11-17,2020-11-19 08:01:53.217
4,BG,2020-11-17,2020-11-19 08:01:53.333
5,CNYUSD=X,2020-11-18,2020-11-19 09:08:23.910
6,EURUSD=X,2020-11-18,2020-11-19 08:01:53.403
7,GC=F,2020-11-18,2020-11-19 08:01:53.480
8,GLEN.L,2020-11-18,2020-11-19 08:01:53.570
9,SI=F,2020-11-18,2020-11-19 08:01:53.640


In [14]:
#Notes:
#https://finance.yahoo.com/quote/usdcad=x/